<a href="https://colab.research.google.com/github/emranahmed72/imran_handsOn_YOLOv11_Car_Object_Detection/blob/main/imran_YOLOv11_Car_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
sshikamaru_car_object_detection_path = kagglehub.dataset_download('sshikamaru/car-object-detection')

print('Data source import complete.')


Data source import complete.


# 1. Install and Import Dependencies

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install ultralytics -q

import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import yaml
from ultralytics import YOLO

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/car-object-detection/data/sample_submission.csv
/kaggle/input/car-object-detection/data/train_solution_bounding_boxes (1).csv
/kaggle/input/car-object-detection/data/testing_images/vid_5_26420.jpg
/kaggle/input/car-object-detection/data/testing_images/vid_5_31520.jpg
/kaggle/input/car-object-detection/data/testing_images/vid_5_29420.jpg
/kaggle/input/car-object-detection/data/testing_images/vid_5_31720.jpg
/kaggle/input/car-object-detection/data/testing_images/vid_5_26560.jpg
/kaggle/input/car-object-detection/data/testing_images/vid_5_27840.jpg
/kaggle/input/car-object-detection/data/testing_images/vid_5_28560.jpg
/kaggle/input/car-object-detection/data/testing_images/vid_5_28360.jpg
/kaggle/input/car-object-detection/data/testing_images/vid_5_25240.jpg
/kaggle/input/car-object-detection/data/testing_images/vid_5_29560.jpg
/kaggle/input/car-object-detection/data/testing_images/vid_5_29020.jpg
/kaggle/input/car-object-detection/data/testing_images/vid_5_30720.jpg
/kaggle/

# 2. Define Paths

In [ ]:
# Original dataset locations
train_img_dir = "/kaggle/input/car-object-detection/data/training_images"
test_img_dir = "/kaggle/input/car-object-detection/data/testing_images"
labels_csv = "/kaggle/input/car-object-detection/data/train_solution_bounding_boxes (1).csv"


In [ ]:
# New YOLOv11-compatible dataset directories
root_dir = "/kaggle/working/data"
os.makedirs(root_dir, exist_ok=True)
os.makedirs(f"{root_dir}/images/train", exist_ok=True)
os.makedirs(f"{root_dir}/images/val", exist_ok=True)
os.makedirs(f"{root_dir}/labels/train", exist_ok=True)
os.makedirs(f"{root_dir}/labels/val", exist_ok=True)

 # 3. Convert CSV Annotations to YOLO Format

In [ ]:
df = pd.read_csv(labels_csv)
df['class'] = 0  # Single class: car
df.rename(columns={'image': 'img_name'}, inplace=True)

# Image size from dataset
W, H = 676, 380

df["x_center"] = ((df["xmin"] + df["xmax"]) / 2) / W
df["y_center"] = ((df["ymin"] + df["ymax"]) / 2) / H
df["width"] = (df["xmax"] - df["xmin"]) / W
df["height"] = (df["ymax"] - df["ymin"]) / H

df_yolo = df[["img_name", "class", "x_center", "y_center", "width", "height"]]


# 4. Train/Val Split and File Preparation

In [ ]:
image_list = sorted(os.listdir(train_img_dir))
np.random.shuffle(image_list)

split = int(0.8 * len(image_list))
train_imgs = image_list[:split]
val_imgs = image_list[split:]

for img_name in image_list:
    subset = "train" if img_name in train_imgs else "val"

    if img_name in df_yolo["img_name"].values:
        rows = df_yolo[df_yolo["img_name"] == img_name]
        label_path = f"{root_dir}/labels/{subset}/{img_name.replace('.jpg','.txt')}"
        with open(label_path, 'w') as f:
            for _, row in rows.iterrows():
                f.write(f"{int(row['class'])} {row['x_center']} {row['y_center']} {row['width']} {row['height']}\n")

    # Copy image
    shutil.copy(f"{train_img_dir}/{img_name}", f"{root_dir}/images/{subset}/{img_name}")


# 5. Create YAML Config File

In [ ]:
data_yaml = {
    'path': root_dir,
    'train': f"{root_dir}/images/train",
    'val': f"{root_dir}/images/val",
    'nc': 1,
    'names': ['car']
}

with open("/kaggle/working/yolo.yaml", 'w') as f:
    yaml.dump(data_yaml, f)


# 6. Train YOLOv11

In [ ]:
model = YOLO("yolo11n.pt")

model.train(
    data="/kaggle/working/yolo.yaml",
    epochs=1,
    imgsz=640,
    batch=8,
    name="car_detector"
)


Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=car_detector4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=

train: Scanning /kaggle/working/data/labels/train... 343 images, 614 backgrounds, 0 corrupt: 100%|██████████| 957/957 [00:00<00:00, 2818.93it/s]

train: New cache created: /kaggle/working/data/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1339.4±243.4 MB/s, size: 89.7 KB)



val: Scanning /kaggle/working/data/labels/val... 126 images, 232 backgrounds, 0 corrupt: 100%|██████████| 358/358 [00:00<00:00, 2809.93it/s]

val: New cache created: /kaggle/working/data/labels/val.cache


...

        1/1         0G      1.436      3.808      1.148          6        640: 100%|██████████| 120/120 [13:41<00:00,  6.84s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [01:00<00:00,  2.63s/it]


                   all        358        183      0.704      0.699      0.734      0.387

1 epochs completed in 0.246 hours.
Optimizer stripped from runs/detect/car_detector4/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/car_detector4/weights/best.pt, 5.4MB

Validating runs/detect/car_detector4/weights/best.pt...
Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 23/23 [00:53<00:00,  2.33s/it]


                   all        358        183      0.704      0.699      0.733      0.387
Speed: 1.6ms preprocess, 135.5ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to runs/detect/car_detector4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b1362b4fc50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

# 7. Evaluate Performance

In [ ]:
model = YOLO("/kaggle/working/runs/detect/car_detector/weights/best.pt")
metrics = model.val()
print("mAP@0.5:", metrics.box.map1)
print("mAP@0.5:0.95:", metrics.box.map)


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/runs/detect/car_detector/weights/best.pt'

# 8. Inference on Test Images

In [ ]:
test_results = model.predict(source=test_img_dir, conf=0.5, iou=0.5, save=False)

# Save predictions
pred_dir = "/kaggle/working/yolov11_results"
img_save_dir = f"{pred_dir}/images"
label_save_dir = f"{pred_dir}/labels"
os.makedirs(img_save_dir, exist_ok=True)
os.makedirs(label_save_dir, exist_ok=True)

for r in test_results:
    name = os.path.basename(r.path).split('.')[0]
    boxes = r.boxes.xyxy.cpu().numpy()
    confs = r.boxes.conf.cpu().numpy()

    label_path = os.path.join(label_save_dir, f"{name}.txt")
    with open(label_path, "w") as f:
        for score, box in zip(confs, boxes):
            f.write(f"{score:.4f} " + " ".join(map(str, box)) + "\n")


# 9. Visualize Predictions

In [ ]:
def show_predictions(img_path, label_path, ax, color=(0, 255, 0)):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Image not found: {img_path}")
        return
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    h, w = img.shape[:2]

    # Skip if label file is empty
    if not os.path.exists(label_path) or os.stat(label_path).st_size == 0:
        ax.imshow(img)
        ax.set_title("No Predictions")
        ax.axis("off")
        return

    labels = pd.read_csv(label_path, sep=' ', header=None).values

    for label in labels:
        score, x1, y1, x2, y2 = label
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, f"{score:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    ax.imshow(img)
    ax.axis("off")


# Plot 10 test images
import random
from matplotlib import pyplot as plt

fig, axes = plt.subplots(5, 2, figsize=(12, 20))
axes = axes.flatten()
sample_imgs = random.sample(os.listdir(test_img_dir), 10)

for i, img_file in enumerate(sample_imgs):
    name = img_file.replace(".jpg", "")
    img_path = os.path.join(test_img_dir, img_file)
    label_path = os.path.join(label_save_dir, f"{name}.txt")
    show_predictions(img_path, label_path, axes[i])
plt.tight_layout()
plt.show()
